# Preparing Bot Dataset 

Import dataset

In [31]:
import pandas as pd

df_friday3 = pd.read_csv("/users/mik2002/aws-dataset/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv")

In [32]:
df_dataset = df_friday3
df_dataset["Label"].value_counts()

Label
Benign    762384
Bot       286191
Name: count, dtype: int64

In [33]:
# Impostazione dell'opzione per mostrare tutte le colonne
pd.set_option('display.max_columns', None)

# Visualizzazione delle prime righe del DataFrame con tutte le colonne
df_dataset.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,6,02/03/2018 08:47:38,141385,9,7,553,3773.0,202,0,61.444444,87.534438,1460,0,539.000000,655.432936,30597.30523,113.166178,9425.666667,19069.116850,73403.0,1.0,141385.0,17673.125,23965.32327,73403.0,22.0,51417.0,8569.50000,13036.89082,31525.0,1.0,0,0,0,0,192,152,63.655975,49.510203,0,1460,254.470588,474.712955,225352.389700,0,0,1,1,0,0,0,1,0,270.375000,61.444444,539.000000,0,0,0,0,0,0,9,553,7,3773,8192,119,4,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,49684,6,02/03/2018 08:47:38,281,2,1,38,0.0,38,0,19.000000,26.870058,0,0,0.000000,0.000000,135231.31670,10676.156580,140.500000,174.655375,264.0,17.0,281.0,281.000,0.00000,281.0,281.0,0.0,0.00000,0.00000,0.0,0.0,1,0,0,0,40,20,7117.437722,3558.718861,0,38,19.000000,21.939310,481.333333,0,1,0,0,1,0,0,0,0,25.333333,19.000000,0.000000,0,0,0,0,0,0,2,38,1,0,123,0,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,443,6,02/03/2018 08:47:40,279824,11,15,1086,10527.0,385,0,98.727273,129.392497,1460,0,701.800000,636.314186,41501.08640,92.915547,11192.960000,24379.448340,112589.0,1.0,279728.0,27972.800,36167.74032,112589.0,94.0,258924.0,18494.57143,36356.50372,133669.0,1.0,0,0,0,0,232,312,39.310424,53.605123,0,1460,430.111111,566.234209,320621.179500,0,0,1,1,0,0,0,1,1,446.653846,98.727273,701.800000,0,0,0,0,0,0,11,1086,15,10527,8192,1047,5,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443,6,02/03/2018 08:47:40,132,2,0,0,0.0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.00000,15151.515150,132.000000,0.000000,132.0,132.0,132.0,132.000,0.00000,132.0,132.0,0.0,0.00000,0.00000,0.0,0.0,0,0,0,0,40,0,15151.515150,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,2,0,0,0,256,-1,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443,6,02/03/2018 08:47:41,274016,9,13,1285,6141.0,517,0,142.777778,183.887722,1460,0,472.384615,611.180489,27100.60726,80.287282,13048.380950,26311.627030,114077.0,1.0,273946.0,34243.250,37996.56546,114077.0,201.0,252994.0,21082.83333,39075.73819,135611.0,1.0,0,0,0,0,192,272,32.844797,47.442485,0,1460,322.869565,497.254764,247262.300400,0,0,1,1,0,0,0,1,1,337.545455,142.777778,472.384615,0,0,0,0,0,0,9,1285,13,6141,8192,1047,5,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


Dropping missing or duplicate values

In [34]:
df_dataset# dropping missing or duplicate values
print("Before drop duplicate and missing values",df_dataset.shape)
df_dataset.drop_duplicates(inplace=True)
print("After drop duplicate values",df_dataset.shape)
df_dataset.dropna(inplace=True)
print("After drop missing values",df_dataset.shape)

# Creazione di una maschera booleana per identificare le righe con valore "Label" nella colonna "Label"
mask = df_dataset["Label"] == "Label"

# Rimozione delle righe identificate dalla maschera
df_dataset = df_dataset.drop(df_dataset[mask].index)


Before drop duplicate and missing values (1048575, 80)
After drop duplicate values (1043116, 80)
After drop missing values (1040558, 80)


Replacing specific DoS attack labels with a general "Dos" category

Creating dummies based on the assumption that the Protocol values are categorical

In [35]:
# Forcing them to be strings
df_dataset = df_dataset.astype({"Protocol": str})
df_dataset = pd.get_dummies(df_dataset, columns=['Protocol'], drop_first=True)
df_dataset["Protocol_17"] = df_dataset["Protocol_17"].astype(int)
df_dataset["Protocol_6"] = df_dataset["Protocol_6"].astype(int)
# making Label column the last column again
df_dataset.insert(len(df_dataset.columns)-1, 'Label', df_dataset.pop('Label'))

Dropping some columns that may be irrelevant (would require re-analysis)

In [36]:
columns_to_drop = [
    'Dst Port',
    'Timestamp',
    'Fwd PSH Flags',
    'Bwd PSH Flags',
    'Fwd URG Flags',
    'Bwd URG Flags',
    'Flow Byts/s',  
    'Flow Pkts/s',
    'Fwd Byts/b Avg',
    'Fwd Pkts/b Avg',
    'Fwd Blk Rate Avg',
    'Bwd Byts/b Avg',
    'Bwd Pkts/b Avg',
    'CWE Flag Count',
    'Bwd Blk Rate Avg'  
]

In [37]:
df_dataset.drop(columns=columns_to_drop, inplace=True)
df_dataset.shape

(1040558, 66)

View the percentages of dataset

In [38]:
display(df_dataset["Label"].value_counts())
df_dataset["Label"].value_counts(normalize=True)

Label
Benign    758248
Bot       282310
Name: count, dtype: int64

Label
Benign    0.728694
Bot       0.271306
Name: proportion, dtype: float64

Saving dataset

In [39]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1040558 entries, 0 to 1048574
Data columns (total 66 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Flow Duration      1040558 non-null  int64  
 1   Tot Fwd Pkts       1040558 non-null  int64  
 2   Tot Bwd Pkts       1040558 non-null  int64  
 3   TotLen Fwd Pkts    1040558 non-null  int64  
 4   TotLen Bwd Pkts    1040558 non-null  float64
 5   Fwd Pkt Len Max    1040558 non-null  int64  
 6   Fwd Pkt Len Min    1040558 non-null  int64  
 7   Fwd Pkt Len Mean   1040558 non-null  float64
 8   Fwd Pkt Len Std    1040558 non-null  float64
 9   Bwd Pkt Len Max    1040558 non-null  int64  
 10  Bwd Pkt Len Min    1040558 non-null  int64  
 11  Bwd Pkt Len Mean   1040558 non-null  float64
 12  Bwd Pkt Len Std    1040558 non-null  float64
 13  Flow IAT Mean      1040558 non-null  float64
 14  Flow IAT Std       1040558 non-null  float64
 15  Flow IAT Max       1040558 non-null  

In [40]:
df_dataset.to_csv("/users/mik2002/som/dataset-processed/processed_bot_dataset.csv", index=False)